In [ ]:
import cv2
import numpy as np
import os
import glob
from tqdm import tqdm

# ==== Directories ====
input_img_dir  = r"C:\Users\awais\OneDrive\Desktop\Thesis\Iran\IranIris"
input_mask_dir = r"C:\Users\awais\OneDrive\Desktop\Iran_Iris_Segmented_Masks"
output_dir     = r"C:\Users\awais\OneDrive\Desktop\Iran_Iris_Isolated_Enhanced"
os.makedirs(output_dir, exist_ok=True)

image_exts = ('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')

# ---------------- ENHANCEMENT HELPERS (from code 2) ----------------
def high_boost_filter(img, k=1.0):
    """
    High-boost sharpening:
        g = f + k * (f - lowpass(f))
    """
    blur = cv2.GaussianBlur(img, (3, 3), 0)
    mask = cv2.subtract(img, blur)
    boosted = cv2.addWeighted(img, 1.0, mask, k, 0)
    return boosted

def clahe_gamma(img,
                clip_limit=2.0,
                tile_grid_size=(8, 8),
                gamma_range=(0.85, 1.15)):
    """
    Local contrast (CLAHE) + random gamma for rich texture.
    """
    clahe = cv2.createCLAHE(clipLimit=clip_limit,
                            tileGridSize=tile_grid_size)
    c = clahe.apply(img)

    gamma = np.random.uniform(*gamma_range)
    norm = c.astype(np.float32) / 255.0
    g = np.power(norm, gamma)
    g = np.clip(g * 255.0, 0, 255).astype(np.uint8)
    return g

# ---------------- MASK MATCHING (from code 1) ----------------
def find_mask(mask_dir, base_name):
    """
    Finds the first matching mask file for a given base name,
    regardless of extension or suffix (_mask, -mask, etc.).
    """
    search_patterns = [
        f"{base_name}.*",           # same name
        f"{base_name}_mask.*",      # name + _mask
        f"{base_name}-mask.*",      # name + -mask
        f"mask_{base_name}.*",      # prefix mask_
    ]

    for pattern in search_patterns:
        matches = glob.glob(os.path.join(mask_dir, pattern))
        if matches:
            return matches[0]  # return first valid match
    return None

# ---------------- IRIS ISOLATION ----------------
def isolate_iris(img, mask):
    """Keeps only the white area (iris) from the image using the given mask."""
    mask_bin = (mask > 127).astype(np.uint8) * 255
    return cv2.bitwise_and(img, img, mask=mask_bin)

# ---------------- MAIN PIPELINE ----------------
processed = 0
missing_masks = 0
skipped_invalid = 0

all_imgs = [f for f in os.listdir(input_img_dir) if f.lower().endswith(image_exts)]

for img_name in tqdm(all_imgs, desc="Isolate iris + enhance"):
    base_name = os.path.splitext(img_name)[0]
    img_path = os.path.join(input_img_dir, img_name)

    mask_path = find_mask(input_mask_dir, base_name)
    if not mask_path:
        missing_masks += 1
        print(f"⚠️ Mask not found for {img_name}")
        continue

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if img is None or mask is None:
        skipped_invalid += 1
        print(f"⚠️ Skipped {img_name} (invalid img/mask file)")
        continue

    # 1) Isolate iris:
    iris_only = isolate_iris(img, mask)

    # 2) Enhance:
    sharp = high_boost_filter(iris_only, k=1.0)
    enhanced = clahe_gamma(
        sharp,
        clip_limit=2.0,
        tile_grid_size=(8, 8),
        gamma_range=(0.85, 1.15)
    )

    # 3) Save
    out_path = os.path.join(output_dir, f"{base_name}.png")
    cv2.imwrite(out_path, enhanced)

    processed += 1

print("\n🎯 Done.")
print("✅ Processed:", processed)
print("⚠️ Missing masks:", missing_masks)
print("⚠️ Invalid/Unreadable skipped:", skipped_invalid)
print("📁 Saved to:", output_dir)


Isolate iris + enhance: 100%|██████████| 792/792 [00:12<00:00, 61.80it/s]


🎯 Done.
✅ Processed: 792
⚠️ Missing masks: 0
⚠️ Invalid/Unreadable skipped: 0
📁 Saved to: C:\Users\awais\OneDrive\Desktop\Iran_Iris_Isolated_Enhanced
